<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#循环神经网络-(recurrent-neural-network,-RNN)" data-toc-modified-id="循环神经网络-(recurrent-neural-network,-RNN)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>循环神经网络 (recurrent neural network, RNN)</a></span></li><li><span><a href="#随时间反向传播-(BPTT)" data-toc-modified-id="随时间反向传播-(BPTT)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>随时间反向传播 (BPTT)</a></span></li><li><span><a href="#长短时记忆网络-(LTSM)" data-toc-modified-id="长短时记忆网络-(LTSM)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>长短时记忆网络 (LTSM)</a></span><ul class="toc-item"><li><span><a href="#遗忘门" data-toc-modified-id="遗忘门-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>遗忘门</a></span></li><li><span><a href="#输入门" data-toc-modified-id="输入门-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>输入门</a></span></li><li><span><a href="#输出门" data-toc-modified-id="输出门-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>输出门</a></span></li></ul></li><li><span><a href="#循环神经网络的变种" data-toc-modified-id="循环神经网络的变种-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>循环神经网络的变种</a></span><ul class="toc-item"><li><span><a href="#双向循环神经网络-(bidirectional-RNN)" data-toc-modified-id="双向循环神经网络-(bidirectional-RNN)-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>双向循环神经网络 (bidirectional RNN)</a></span></li><li><span><a href="#深层循环神经网络-(deepRNN)" data-toc-modified-id="深层循环神经网络-(deepRNN)-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>深层循环神经网络 (deepRNN)</a></span></li><li><span><a href="#循环神经网络的dropout" data-toc-modified-id="循环神经网络的dropout-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>循环神经网络的dropout</a></span></li></ul></li><li><span><a href="#利用RNN训练手写数字MNIST数据集" data-toc-modified-id="利用RNN训练手写数字MNIST数据集-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>利用RNN训练手写数字MNIST数据集</a></span><ul class="toc-item"><li><span><a href="#tf.nn.dynamic_rnn-函数说明" data-toc-modified-id="tf.nn.dynamic_rnn-函数说明-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>tf.nn.dynamic_rnn 函数说明</a></span></li></ul></li><li><span><a href="#利用RNN进行回归训练" data-toc-modified-id="利用RNN进行回归训练-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>利用RNN进行回归训练</a></span></li></ul></div>

## 循环神经网络 (recurrent neural network, RNN)
[Link1](https://www.imooc.com/article/23821)


> 循环神经网络的主要用途是处理和预测`序列数据`。循环神经网络的隐藏层之间的结点是有连接的，隐藏层的输入不仅包括输入层的输出，还包括上一时刻隐藏层的输出。

> 循环神经网络会对于每一个时刻的输入结合当前模型的状态给出一个输出。循环神经网络在每一个时刻会有一个输入X(t)，然后根据循环神经网络当前的状态A(t)提供一个输出H(t)；而循环神经网络当前的状态A(t)是根据上一时刻的状态A(t-1)和当前的输入X(t)共同决定的。

> 对于一个序列数据，可以将这个序列上不同时刻的数据依次传入循环神经网络的输入层，而输出可以是对序列中下一个时刻的预测，也可以是对当前时刻信息的处理结果。

> 在循环神经网络中，循环体网络结构中的参数`在不同时刻是共享的`。

> 循环神经网络的总损失为所有时刻(或者部分时刻)上的损失函数的总和。

![RNN前向传播计算示意图](illustration/RNN前向传播计算示意图.png)

## 随时间反向传播 (BPTT)
由于RNN是一种基于时序数据的神经网络模型，因此传统的BP算法并不适用于该模型的优化；在RNN中最常用的优化算法是`随时间反向传播`(BackPropagation Through Time)  

<br/>在RNN网络中，总体错误是每一时刻的错误的累加和。我们的目标是计算错误值相对于参数U, V, W的梯度以及用随机梯度下降学习好的参数；为了计算梯度，使用的是链式求导法则，主要是用反向传播算法往后传播错误。

![BPTT示意图](illustration/BPTT示意图.png)

> * 绿色箭头表示的是时间上的反向传播过程
> * 红色箭头表示的是同一时刻空间上的传播过程 (也就是普通前馈神经网络的误差过程)

BPTT算法存在的问题
*  梯度消失 (gradient vanishing）
>  有效的解决方案有：合适的初始化矩阵W可以减小梯度消失效应，正则化也能起作用。更好的方法是选择ReLU而不是sigmoid和tanh作为激活函数。ReLU的导数是常数值0或1，所以不可能会引起梯度消失。更通用的方案时采用长短项记忆（LSTM）或门限递归单元（GRU）结构  
> LSTM只能避免RNN的梯度消失（gradient vanishing）

*  梯度爆炸 (gradient explosion)
>  一般靠裁剪后的优化算法即可解决，比如gradient clipping（如果梯度的范数大于某个给定值，将梯度同比收缩）

## 长短时记忆网络 (LTSM)

循环神经网络工作的关键点是使用历史的信息来帮助当前的决策；但同时也带来了更大的技术挑战 -- 长期依赖(long-term dependencies)

<br/>LSTM的提出就是为了解决长期依赖问题。LSTM是一种拥有三个"门"结构的特殊网络结构。(所谓"门"的结构就是一个使用sigmoid神经网络和和一个按位做乘法的操作，这两个操作合在一起就是一个"门"结构，之所以该结构叫做"门"是因为使用sigmoid作为激活函数的全连接神经网络层会输出一个0到1之间的数值，描述当前输入有多少信息量可以通过这个结构)

### 遗忘门
LSTM 中的第一步是决定会从细胞状态中丢弃什么信息。这个决定通过一个称为忘记门层完成。该门会读取`h(t-1)`和`x(t)`，输出一个在 0到 1之间的数值给每个在细胞状态 `C(t-1)` 中的数字。1 表示“完全保留”，0 表示“完全舍弃”。
![forget_gate](illustration/forget_gate.jpg)

> 其中`h(t-1)`表示的是上一个cell的输出, `x(t)`表示的是当前cell的输入, σ 表示sigmod函数


### 输入门
LSTM 中的第二步是决定让多少新的信息加入到 cell 状态中来。实现这个需要包括两个步骤：首先，一个叫做“input gate layer ”的 sigmoid 层决定哪些信息需要更新；一个 tanh 层生成一个向量，也就是备选的用来更新的内容。在下一步，把这两部分联合起来，对 cell 的状态进行一个更新
![input_gate](illustration/input_gate.jpg)

<br/>现在是更新旧细胞状态的时间了，目标是将`C(t-1)`更新为`C(t)`。 把旧状态与`f(t)`相乘，丢弃掉确定需要丢弃的信息；接着加上`i(t)` * `C^(t)`，这就是新的候选值，这两者决定了每个状态更新情况
![input_gate_02](illustration/input_gate_02.jpg)


### 输出门
LSTM 中的第三步是决定输出什么值。这个输出将会基于 cell 的状态，但是也是一个过滤后的版本。首先，运行一个 sigmoid 层来确定细胞状态的哪个部分将输出出去。接着，把细胞状态通过 tanh 进行处理（得到一个在 -1 到 1 之间的值）并将它和 sigmoid 门的输出相乘，最终仅仅会输出确定输出的那部分。
![output_gate](illustration/output_gate.jpg)


In [ ]:
# 定义一个LSTM结构, 并指定隐藏层的大小
lstm = tf.contrib.rnn.BasicLSTMCell(lstm_hidden_size)
'''
__init__(
    num_units,
    forget_bias=1.0,
    state_is_tuple=True,
    activation=None,
    reuse=None,
    name=None,
    dtype=None
)
'''

# 将LSTM中的状态初始化为全0数组. 和其他神经网络类似, 在优化神经网络时, 
# 每次也会使用一个batch的训练样本, 以下代码中, batch_size给出了一个batch大小
# BasicLSTMCell类提供了zero_state函数来生成全0的初始状态
state = lstm.zero_state(batch_size, tf.float32)
'''
zero_state(batch_size, dtype)
Args:
    batch_size:  int, float, or unit Tensor representing the batch size.
    dtype:  the data type to use for the state.
Return:
    zero-filled state tensor(s).    
'''

# 虽然理论上循环神经网络可以处理任意长度的序列, 但是在训练时为了避免梯度消散的问题,
# 通常会规定一个最大的序列长度, 以下代码中, 用 num_steps 来表示这个长度
for i in range(num_steps):
    # 在第一个时刻声明LSTM结构中使用的变量
    # 在之后的时刻都需要复用之前定义好的变量
    if i > 0:
        tf.get_variable_scope().reuse_variables()
    # 每一步处理时间序列中的一个时刻。将当前输入(current_input)和前一时刻状态(state)
    # 传入定义的LSTM结构可以得到当前LSTM结构的输出lstm_output和更新后的状态state
    lstm_output, state = lstm(current_input, state)
    # 将当前时刻LSTM结构的输出传入一个全连接层得到最后的输出
    final_output = fully_connected(lstm_output)
    

## 循环神经网络的变种

### 双向循环神经网络 (bidirectional RNN)

双向循环神经网络是由两个循环神经网络上下叠加在一起组成的。输出由这两个循环神经网络的状态共同决定。  
![双向循环神经网络结构示意图](illustration/双向循环神经网络结构示意图.png)  

> 双向循环神经网络的主体结构就是两个单向循环神经网络的结合。在每一个时刻`t`, 输入会同时提供给这两个方向相反的循环神经网络, 而输出则是由这两个单向循环神经网络共同决定。双向循环神经网络的前向传播过程和单向的循环神经网络十分类似

### 深层循环神经网络 (deepRNN)
深层循环神经网络在每一个时刻上将循环体结构复制了多次，每一层的循环体中参数是一致的，而不同层中的参数可以不同   
![深层循环神经网络结构示意图](illustration/深层循环神经网络结构示意图.png)

### 循环神经网络的dropout
在循环神经网络中使用`dropout`可以使网络更加健壮；区别于卷积神经网络只在最后的全连接层中使用`dropout`, 循环神经网络一般只在不同层循环体结构之间使用`dropout`, 而不在同一层的循环体结构之间使用。也就是说从时刻`t-1`传递到时刻`t`时，循环神经网络不会进行状态的`dropout`；而在同一时刻`t`中，不同层循环体之间会使用`dropout`  
![深层循环神经网络使用dropout示意图](illustration/深层循环神经网络使用dropout示意图.png)

> 图中实线箭头表示不使用`dropout`，虚线箭头表示使用`dropout`

## 利用RNN训练手写数字MNIST数据集
[Link1](https://blog.csdn.net/omnispace/article/details/78415100)

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
tf.set_random_seed(1)   # set random seed
 
# 导入数据
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
 
# hyperparameters
lr = 0.001                         # learning rate
training_iters = 100000     # train step 上限
batch_size = 128            
n_inputs = 28                  # MNIST data input (img shape: 28*28)
n_steps = 28                   # time steps
n_hidden_units = 128       # neurons in hidden layer
n_classes = 10                # MNIST classes (0-9 digits)

# x y placeholder
x = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_classes])
 
# 对 weights biases 初始值的定义
weights = {
    # shape (28, 128)
    'in': tf.Variable(tf.random_normal([n_inputs, n_hidden_units])),
    # shape (128, 10)
    'out': tf.Variable(tf.random_normal([n_hidden_units, n_classes]))
}
biases = {
    # shape (128, )
    'in': tf.Variable(tf.constant(0.1, shape=[n_hidden_units, ])),
    # shape (10, )
    'out': tf.Variable(tf.constant(0.1, shape=[n_classes, ]))
}

'''
  RNN 主体总共有3部分组成(input_layer[输入隐藏层],  rnn_cell[rnn网络层],  output_layer[输出隐藏层])
'''
def RNN(X, weights, biases):
    # 定义 input_layer 层...
    # 原始的 X 是 3 维数据, 我们需要把它变成 2 维数据才能使用 weights 的矩阵乘法
    # X ==> (128 batches * 28 steps, 28 inputs)
    X = tf.reshape(X, [-1, n_inputs])
    # X_in = W*X + b
    X_in = tf.matmul(X, weights['in']) + biases['in']
    # X_in ==> (128 batches, 28 steps, 128 hidden) 换回3维
    X_in = tf.reshape(X_in, [-1, n_steps, n_hidden_units])

    # 定义 rnn_cell 层...
    # 使用 basic LSTM Cell.
    # 对于 lstm 来说, state可被分为(c_state, h_state).
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_units, forget_bias=1.0, state_is_tuple=True)
    # 初始化全零 state
    init_state = lstm_cell.zero_state(batch_size, dtype=tf.float32) 
    # 如果 inputs 为 (batches, steps, inputs) ==> time_major=False;
    # 如果 inputs 为 (steps, batches, inputs) ==> time_major=True;
    outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, X_in, initial_state=init_state, time_major=False)

    # 定义 output_layer 层...
    # 由于该例的特殊性, 有两种方法来计算 output_layer 输出
    # 方式一: 直接调用final_state 中的 h_state ( 通过final_state[1]获取) 来进行运算:
    results = tf.matmul(final_state[1], weights['out']) + biases['out']  
    # 方式二: 调用最后一个 outputs (在这个例子中, 和上面的final_state[1]是一样的):
    # 由于time_major=False,  故outputs 的形状为 [batch_size, max_time, cell.output_size]
    # 把 outputs 变成 列表 [(batch, outputs)..] * steps
    outputs = tf.unstack(tf.transpose(outputs, [1,0,2]))
    results = tf.matmul(outputs[-1], weights['out']) + biases['out']    #选取最后一个 output

return results

# 定义 cost 和 train_op:
pred = RNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
train_op = tf.train.AdamOptimizer(lr).minimize(cost)

# 训练 RNN
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
init = tf.global_variables_initializer()
 
with tf.Session() as sess:
    sess.run(init)
    step = 0
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape([batch_size, n_steps, n_inputs])
        sess.run([train_op], feed_dict={
            x: batch_xs,
            y: batch_ys,
        })
        if step % 20 == 0:
            print(sess.run(accuracy, feed_dict={
            x: batch_xs,
            y: batch_ys,
        }))
        step += 1

### tf.nn.dynamic_rnn 函数说明

```python
tf.nn.dynamic_rnn(
    cell,
    inputs,
    sequence_length=None,
    initial_state=None,
    dtype=None,
    parallel_iterations=None,
    swap_memory=False,
    time_major=False,
    scope=None
)
```

Returns:   A pair (outputs, state)
[Link1](https://blog.csdn.net/u010960155/article/details/81707498)

outputs 是一个tensor  
> 如果time_major==True，outputs形状为 [max_time, batch_size, cell.output_size]（要求rnn输入与rnn输出形状保持一致）     

> 如果time_major==False(默认)，outputs形状为 [batch_size, max_time, cell.output_size]
   

state 是一个tensor  
> 如果cell为LSTM, state形状为[2，batch_size, cell.output_size]; 每个cell会有两个输出: `C(t)` 和 `h(t)`，上面这个图是输出`C(t)`，代表哪些信息应该被记住哪些应该被遗忘；下面这个图是输出`h(t)`，代表这个cell的最终输出，LSTM的state是由`C(t)` 和 `h(t)` 组成的
![LSTM_state_示意图](illustration/LSTM_state_示意图.png)

> 如果cell为GRU, state形状为[batch_size, cell.output_size];  state就只有一个了，原因是GRU将`C(t)` 和 `h(t)` 进行了简化，将其合并成了`h(t)`，如下图所示，GRU将遗忘门和输入门合并成了更新门，另外cell不在有细胞状态cell state，只有hidden state
![GRU_state_示意图](illustration/GRU_state_示意图.png)

output 与 state 的关系
> 如果cell为LSTM，那 state是个tuple，分别代表`C(t)` 和 `h(t)` ，其中 `h(t)` 与outputs中的对应的最后一个时刻的输出相等，假设state形状为[2，batch_size, cell.output_size]，outputs形状为 [batch_size, max_time, cell.output_size]，那么`state[1, batch_size, : ]` == `outputs[batch_size, -1, : ]`
```python
'''
由于time_major=False,  故outputs 的形状为 [batch_size, max_time, cell.output_size]
需要将outputs转换成[max_time, batch_size, cell.output_size]形状 [0, 1, 2] => [1, 0, 2]
则：outputs[-1] => [ -1, batch_size , cell.output_size ] => [ -1, : , : ] 即为最后一个cell的输出
'''
outputs = tf.unstack(tf.transpose(outputs, [1,0,2]))
results = tf.matmul(outputs[-1], weights['out']) + biases['out'] 
```

> 如果cell为GRU，同上，state其实就是 `h(t)`，`state` == `outputs[-1]`


Create a BasicRNNCell
```python
# create a BasicRNNCell
rnn_cell = tf.nn.rnn_cell.BasicRNNCell(hidden_size)

# 'outputs' is a tensor of shape [batch_size, max_time, cell_state_size]

# defining initial state
initial_state = rnn_cell.zero_state(batch_size, dtype=tf.float32)

# 'state' is a tensor of shape [batch_size, cell_state_size]
outputs, state = tf.nn.dynamic_rnn(rnn_cell, input_data,
                                   initial_state=initial_state,
                                   dtype=tf.float32)
```

Create 2 LSTMCells
```python
# create 2 LSTMCells
rnn_layers = [tf.nn.rnn_cell.LSTMCell(size) for size in [128, 256]]

# create a RNN cell composed sequentially of a number of RNNCells
multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)

# 'outputs' is a tensor of shape [batch_size, max_time, 256]
# 'state' is a N-tuple where N is the number of LSTMCells containing a
# tf.contrib.rnn.LSTMStateTuple for each cell
outputs, state = tf.nn.dynamic_rnn(cell=multi_rnn_cell,
                                   inputs=data,
                                   dtype=tf.float32)
```

In [ ]:
import numpy as np
import tensorflow as tf

def dynamic_rnn(rnn_type='lstm'):

    # 创建输入数据, 3代表batch size, 6代表输入序列的最大步长(max time), 4代表每个序列的维度
    X = np.random.randn(3, 6, 4) # 返回一个[3, 6, 4]的矩阵

    # 将X[1, 4:]的元素置0，即将第二个输入的实际长度修改为4
    X[1, 4:] = 0

    # 记录三个输入的实际步长 (模拟变长情况)
    X_lengths = [6, 4, 6]

    # 指定RNN隐藏层神经元个数
    rnn_hidden_size = 5

    if rnn_type == 'lstm':
        cell = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_hidden_size)
    else:
        cell = tf.contrib.rnn.GRUCell(num_units=rnn_hidden_size)

    outputs, last_states = tf.nn.dynamic_rnn(
        cell = cell,
        dtype = tf.float64,
        sequence_length = X_lengths,
        inputs = X,
    )

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        val_output, val_states = sess.run([outputs, last_states])
        print np.shape(val_output)       # LSTM: (3, 6, 5);  GRU: (3, 6, 5)
        print np.shape(val_states)       # LSTM: (2, 6, 5);  GRU: (3, 5)

"""
cell类型为LSTM时，输入的形状为 [3, 6, 4]，经过 `tf.nn.dynamic_rnn` 后outputs的形状为[3, 6, 5]，
state形状为 [2, 3, 5], 其中state第一部分为c, 代表 `cell state`; 第二部分为h，代表 `hidden state`。
可以看到 `hidden state` 与对应的 `outputs` 的最后一行是相等的。
另外需要注意的是输入一共有三个序列，但第二个序列的长度只有4，可以看到outputs中对应的两行值都为0，
所以hidden state对应的是最后一个不为0的部分。tf.nn.dynamic_rnn通过设置`sequence_length`来实现这一逻辑


cell类型为GRU时，输入的形状为 [3, 6, 4]，经过 `tf.nn.dynamic_rnn` 后outputs的形状为 [3, 6, 5]，
state形状为 [3, 5]。可以看到 state 与对应的 outputs 的最后一行是相等的
"""                

if __name__ == '__main__':
    dynamic_rnn('lstm')    
    dynamic_rnn('gru')

## 利用RNN进行回归训练

In [ ]:
#-*- coding:utf-8 -*-

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


BATCH_START = 0
TIME_STEPS = 20
BATCH_SIZE = 50
INPUT_SIZE = 1
OUTPUT_SIZE = 1
CELL_SIZE = 10      # LSTM神经元个数
LR = 0.006


def get_batch():
    global BATCH_START, TIME_STEPS
    # xs shape (50batch, 20steps)
    xs = np.arange(BATCH_START, BATCH_START+TIME_STEPS*BATCH_SIZE).reshape((BATCH_SIZE, TIME_STEPS)) / (10*np.pi)
    seq = np.sin(xs)
    res = np.cos(xs)
    BATCH_START += TIME_STEPS
    # seq[:, :, np.newaxis].shape = (50, 20, 1)
    # res[:, :, np.newaxis].shape = (50, 20, 1)
    return [seq[:, :, np.newaxis], res[:, :, np.newaxis], xs]




class LSTMRNN(object):

    def __init__(self, n_steps, input_size, output_size, cell_size, batch_size):
        self.n_steps = n_steps
        self.input_size = input_size
        self.output_size = output_size
        self.cell_size = cell_size
        self.batch_size = batch_size

        self.l_in_y = None
        self.cell_init_state = None
        self.cell_outputs = None
        self.cell_final_state = None
        self.pred = None
        self.cost = None

        with tf.name_scope('inputs'):
            self.xs = tf.placeholder(tf.float32, [None, n_steps, input_size], name='xs')
            self.ys = tf.placeholder(tf.float32, [None, n_steps, output_size], name='ys')
        with tf.variable_scope('in_hidden'):
            self.add_input_layer()
        with tf.variable_scope('LSTM_cell'):
            self.add_lstm_cell()
        with tf.variable_scope('out_hidden'):
            self.add_output_layer()
        with tf.name_scope('cost'):
            self.compute_cost()
        with tf.name_scope('train'):
            self.train_op = tf.train.AdamOptimizer(LR).minimize(self.cost)


    @staticmethod
    def ms_error(labels, logits):
        return tf.square(tf.subtract(labels, logits))

    @staticmethod
    def weight_variable(shape, name='weights'):
        initializer = tf.random_normal_initializer(mean=0., stddev=1.,)
        return tf.get_variable(shape=shape, initializer=initializer, name=name)

    @staticmethod
    def bias_variable(shape, name='biases'):
        initializer = tf.constant_initializer(0.1)
        return tf.get_variable(name=name, shape=shape, initializer=initializer)



    def add_input_layer(self):

        # 将3D数据转换成2D, 用于输入隐藏层(2D)计算
        l_in_x = tf.reshape(self.xs, [-1, self.input_size], name='2_2D')
        Ws_in = self.weight_variable([self.input_size, self.cell_size])
        bs_in = self.bias_variable([self.cell_size,])

        with tf.name_scope("Wx_plus_b"):
            l_in_y = tf.matmul(l_in_x, Ws_in) + bs_in

        # 将计算结果转换为3D, 用于lstm计算
        self.l_in_y = tf.reshape(l_in_y, [-1, self.n_steps, self.cell_size], name='2_3D')


    def add_lstm_cell(self):

        lstm_cell = tf.contrib.rnn.BasicLSTMCell(self.cell_size)

        with tf.name_scope("initial_state"):
            self.cell_init_state = lstm_cell.zero_state(self.batch_size, dtype=tf.float32)

        self.cell_outputs, self.cell_final_state = tf.nn.dynamic_rnn(
            lstm_cell, self.l_in_y, initial_state=self.cell_init_state, time_major=False
        )


    def add_output_layer(self):

        # 将3D数据转换成2D, 用于输入隐藏层(2D)计算
        # time_major=False => self.cell_outputs = [batch_size, max_time, cell.output_size]
        # l_out_x = [(batch_size * max_time), cell.output_size]
        # 注意区别 (计算损失函数的不同导致 outputs 的取值变化)
        # outputs = tf.unstack(tf.transpose(outputs, [1,0,2]))
        # tf.matmul(outputs[-1], weights['out']) + biases['out']

        l_out_x = tf.reshape(self.cell_outputs, [-1, self.cell_size], name='2_2D')
        Ws_out = self.weight_variable([self.cell_size, self.output_size])
        bs_out = self.bias_variable([self.output_size, ])

        with tf.name_scope('Wx_plus_b'):
            # np.shape(self.pred) = (1000, 1) => [(BATCH_SIZE * TIME_STEPS), OUTPUT_SIZE]
            self.pred = tf.matmul(l_out_x, Ws_out) + bs_out


    def compute_cost(self):

        """
            tf.contrib.legacy_seq2seq.sequence_loss_by_example(
                logits,
                targets,
                weights,
                average_across_timesteps=True,
                softmax_loss_function=None,
                name=None
            )

            # Weighted cross-entropy loss for a sequence of logits (per example).
            # 这个函数用于计算所有example的加权交叉熵损失
            # 该函数的返回值是一个1D float类型的Tensor，尺寸为`batch_size`，
            # 其中的每一个元素代表当前输入序列example的交叉熵

            # logits: List of 2D Tensors of shape [batch_size x num_decoder_symbols].
            # targets: List of 1D batch-sized int32 Tensors of the same length as logits
            # weights: List of 1D batch-sized float-Tensors of the same length as logits.
            # average_across_timesteps: If set, divide the returned cost by the total label weight.
            # softmax_loss_function: Function (labels, logits) ->
                    loss-batch to be used instead of the standard softmax (the default if this is None).

            # 结合本例进一步理解:
            #   logits 的shape = [(batch_size * time_step), output_size], output_size 即使(分类)类别的个数
            #   targets 的shape = [(batch_size * time_step)]
            #   sequence_loss_by_example 的做法是, 针对logits中的每一个`time_step`, 即[batch_size, output_size],
            # 对所有`output_size`个预测结果, 得出预测值最大的那个类别, 与target中的值相比较计算`loss`值
        """

        losses = tf.contrib.legacy_seq2seq.sequence_loss_by_example(
            [tf.reshape(self.pred, [-1], name='reshape_pred')],    # shape = (1000, )
            [tf.reshape(self.ys, [-1], name='reshape_target')],    # shape = (1000, ）
            [tf.ones([self.batch_size * self.n_steps], dtype=tf.float32)],
            average_across_timesteps = True,
            softmax_loss_function = self.ms_error,
            name = 'losses'
        )

        # 对`losses`求均值, 返回一个标量
        with tf.name_scope('average_cost'):
            self.cost = tf.div(
                tf.reduce_mean(losses, name='losses_sum'),
                self.batch_size,
                name = 'average_cost'
            )
            tf.summary.scalar('cost', self.cost)


if __name__ == '__main__':

    model = LSTMRNN(TIME_STEPS, INPUT_SIZE, OUTPUT_SIZE, CELL_SIZE, BATCH_SIZE)

    sess = tf.Session()
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter("logs", sess.graph)
    sess.run(tf.global_variables_initializer())

    plt.ion()
    plt.show()

    state = None
    for i in range(10):
        seq, res, xs = get_batch()
        if i == 0:
            feed_dict = {model.xs : seq, model.ys : res}
        else:
            # 动态更新 tf.nn.dynamic_rnn 函数的 initial_state
            feed_dict = {model.xs : seq, model.ys : res, model.cell_init_state : state}

        _, cost, state, pred = sess.run(
            [model.train_op, model.cost, model.cell_final_state, model.pred],
            feed_dict = feed_dict
        )

        plt.plot(xs[0, :], res[0].flatten(), 'r', xs[0, :], pred.flatten()[:TIME_STEPS], 'b--')
        plt.ylim((-1.2, 1.2))
        plt.draw()
        plt.pause(0.3)

        if i % 20 == 0:
            print 'cost: ', round(cost, 4)
            result = sess.run(merged, feed_dict)
            writer.add_summary(result, i)
